In [32]:
import glob
import numpy as np
import scipy.io as so
import os
import cv2 as cv
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from sklearn.model_selection import train_test_split

from utils.model import WideNet, GrayNet
from utils.data import ImageDataset
from utils.signal import standardize_image, read_mat_file, add_noise, signal_to_gray_image, signal_to_scalorgram_image

In [2]:
datapath = './data/transformed_images/hp_0/noise_0_db/'

In [15]:
subfolder = os.listdir(datapath)
print(subfolder)

['Outer7', 'Ball21', 'Outer21', 'Inner7', 'Inner21', 'Inner14', 'Outer14', 'Ball14', 'NoFault']


In [16]:
label_class_dict = {'NoFault': 0, 
                    'Inner7': 1, 'Inner14': 2, 'Inner21': 3, 
                    'Ball7': 4, 'Ball14': 5, 'Ball21': 6,
                    'Outer7': 7, 'Outer14': 8, 'Outer21': 9}

In [17]:
train_list, test_list = [], []

In [18]:
for fname in subfolder:
    list_all_file = glob.glob(datapath + fname + '/gray/*.png')
    _train, _test = train_test_split(list_all_file, test_size=0.2)
    train_list.extend(_train)
    test_list.extend(_test)

In [19]:
print(len(train_list), len(test_list))

2160 540


In [27]:
dataset = ImageDataset(train_list, label_class_dict, ToTensor())

In [28]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [29]:
train_features, train_labels = next(iter(dataloader))

In [33]:
model = GrayNet(10)

GrayNet(
  (gray): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (full): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)